In [8]:
import pandas as pd
import plotly.graph_objects as go

In [17]:
file_path = "./model_risk_signals.csv"
df = pd.read_csv(file_path)
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.rename(columns={'Model_signals': 'model_signals'})


In [1]:
df

NameError: name 'df' is not defined

In [25]:
df['EMA12'] = df['price'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['price'].ewm(span=26, adjust=False).mean()

df['MACD'] = df['EMA12'] - df['EMA26']

df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

df['Histogram'] = df['MACD'] - df['Signal']

df[['price', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram']].head()


,price,EMA12,EMA26,MACD,Signal,Histogram
0,8894,8894.000000,8894.000000,0.000000,0.000000,0.000000
1,8700,8864.153846,8879.629630,-15.475783,-3.095157,-12.380627
2,8464,8802.591716,8848.842250,-46.250534,-11.726232,-34.524302
3,8512,8757.885298,8823.890972,-66.005674,-22.582120,-43.423553
4,8445,8709.749098,8795.824974,-86.075876,-35.280871,-50.795004


In [26]:
initial_capital = 1000
capital = initial_capital
cash = initial_capital
in_trade = False
entry_price = 0
balance_history = [initial_capital]
trades = []

for i in range(1, len(df)):
    # Enter a short trade
    if df['Histogram'].iloc[i-1] >= 0 and df['Histogram'].iloc[i] < 0 and not in_trade:
        in_trade = True
        entry_price = df['price'].iloc[i]
        amount_to_trade = 0.5 * cash
        cash -= amount_to_trade
    
    # Exit the short trade
    elif df['Histogram'].iloc[i-1] < 0 and df['Histogram'].iloc[i] >= 0 and in_trade:
        in_trade = False
        exit_price = df['price'].iloc[i]
        pnl = amount_to_trade * (entry_price - exit_price) / exit_price
        cash += amount_to_trade + pnl
        trades.append(pnl)
    
    # Record balance
    balance_history.append(cash + (amount_to_trade + amount_to_trade * (entry_price - df['price'].iloc[i]) / df['price'].iloc[i]) if in_trade else cash)

# Calculate metrics
total_pnl = sum(trades)
total_roi = (cash - initial_capital) / initial_capital * 100
num_trades = len(trades)
num_profitable_trades = len([trade for trade in trades if trade > 0])
num_losing_trades = len([trade for trade in trades if trade < 0])
winrate = num_profitable_trades / num_trades * 100
max_drawdown = min(balance_history) - max(balance_history)

metrics = {
    'Total ROI': total_roi,
    'Total PnL': total_pnl,
    'Max Drawdown': max_drawdown,
    'Number of Trades': num_trades,
    'Number of Profitable Trades': num_profitable_trades,
    'Number of Losing Trades': num_losing_trades,
    'Winrate': winrate
}

metrics


{'Total ROI': 70.51393502004301,
 'Total PnL': 705.1393502004294,
 'Max Drawdown': -1003.6746783518165,
 'Number of Trades': 71,
 'Number of Profitable Trades': 25,
 'Number of Losing Trades': 45,
 'Winrate': 35.2112676056338}

In [28]:

short_entries = df[(df['Histogram'].shift(1) >= 0) & (df['Histogram'] < 0)]
short_exits = df[(df['Histogram'].shift(1) < 0) & (df['Histogram'] >= 0)]

# Create the main price chart for signals on price
fig1 = go.Figure()

# Add price data
fig1.add_trace(go.Scatter(x=df['Datetime'], y=df['price'], mode='lines', name='Price'))

# Add short entry points
fig1.add_trace(go.Scatter(x=short_entries['Datetime'], y=short_entries['price'], mode='markers', 
                          marker=dict(size=8, color='red', symbol='circle'), name='Short Entry'))

# Add short exit points
fig1.add_trace(go.Scatter(x=short_exits['Datetime'], y=short_exits['price'], mode='markers', 
                          marker=dict(size=8, color='green', symbol='circle'), name='Short Exit'))

# Set title and labels
fig1.update_layout(title='MACD Short Trading Signals on Price', xaxis_title='Date', yaxis_title='Price')

# Show the plot
fig1.show()

# Create the balance over time plot
fig2 = go.Figure()

# Assuming balance_history is already computed from the MACD strategy
# ...

fig2.add_trace(go.Scatter(x=df['Datetime'], y=balance_history, mode='lines', name='Balance'))

# Set title and labels
fig2.update_layout(title='Balance Over Time', xaxis_title='Date', yaxis_title='Balance')

# Show the plot
fig2.show()


In [30]:
fig1 = go.Figure()

# Plotting price data
fig1.add_trace(go.Scatter(x=df['Datetime'], y=df['price'], mode='lines', name='Price'))

# Highlighting short signals
short_signals = df[df['Short_Signal']]
fig1.add_trace(go.Scatter(x=short_signals['Datetime'], y=short_signals['price'], 
                          mode='markers', marker=dict(size=6, color='red', symbol='triangle-down'), 
                          name='Short Signal'))

# Configurations
fig1.update_layout(title='MACD Short Trading Signals on Price Chart',
                   xaxis_title='Date',
                   yaxis_title='Price',
                   template="plotly_dark")

fig1.show()

KeyError: 'Short_Signal'

In [24]:
import plotly.graph_objects as go

# Create the figure for Portfolio Balance Over Time
fig_balance = go.Figure()

fig_balance.add_trace(go.Scatter(x=df['Datetime'], y=portfolio_values, mode='lines', name='Portfolio Balance', line=dict(color='blue')))
fig_balance.update_layout(title='Portfolio Balance Over Time', xaxis=dict(title='Datetime'), yaxis=dict(title='Portfolio Value'))
fig_balance.show()

# Create the figure for Price Over Time
fig_price = go.Figure()

fig_price.add_trace(go.Scatter(x=df['Datetime'], y=df['price'], mode='lines', name='Price', line=dict(color='red')))
fig_price.update_layout(title='Price Over Time', xaxis=dict(title='Datetime'), yaxis=dict(title='Price'))
fig_price.show()
